## Задача 4. Seed-and-Extend

In [7]:
reference = "CTAGGATCCAGGCATACGA"   
query= "GGATCCATTCATTA"        

k = 4   
X_DROP = 2   

MATCH=  1
MISMATCH = -1

def score_pair(a, b):
    return MATCH if a == b else MISMATCH

**Идея алгоритма**

**Seed-and-Extend** — эвристика для быстрого локального выравнивания. Вместо того чтобы считать матрицу Смита–Уотермана за O(n×m), алгоритм работает в две фазы:

**Фаза 1 — Seeding:**  
Разбиваем query на все k-меры (подстроки длины k). Для каждого k-мера ищем его точное вхождение в референс через заранее построенный индекс.  
Каждое найденное совпадение — это **seed** (якорь): пара позиций (i, j).

**Фаза 2 — Extension:**  
От каждого seed расширяемся влево и вправо, добавляя по одному символу. Ведём два счётчика: `Scur` (текущий счёт) и `Smax` (лучший достигнутый). Стоп-критерий — **X-drop**: если `Smax − Scur ≥ X` (задан), расширение прекращается.  
Итоговое выравнивание обрезается по позиции `Smax`.

### Фаза 1: построение индекса и поиск seeds

In [8]:
def build_index(seq, k):
    """
    Строит словарь: k-мер 
    """
    index = {}
    for i in range(len(seq) - k + 1):
        kmer = seq[i : i + k]             
        index.setdefault(kmer, []).append(i)
    return index


def find_seeds(query, db_index, k):
    """
    Для каждого k-мера запроса ищем его в индексе референса.
    Каждое совпадение = seed: (k-мер, позиция в query, позиция в ref).
    """
    seeds = []
    for i in range(len(query) - k + 1):
        kmer = query[i : i + k]
        if kmer in db_index:
            for r_pos in db_index[kmer]:
                seeds.append((kmer, i, r_pos))
    return seeds


db_index = build_index(reference, k)

print("Индекс референса")
for kmer in sorted(db_index):
    print(f"  '{kmer}': позиции {db_index[kmer]}")

seeds = find_seeds(query, db_index, k)

print(f"\n─── Найденные seeds (k={k}) ───")
for kmer, qp, rp in seeds:
    print(f"  '{kmer}':  query[{qp}:{qp+k}]  ==  ref[{rp}:{rp+k}]   стартовый score = {k*MATCH}")

Индекс референса
  'ACGA': позиции [15]
  'AGGA': позиции [2]
  'AGGC': позиции [9]
  'ATAC': позиции [13]
  'ATCC': позиции [5]
  'CAGG': позиции [8]
  'CATA': позиции [12]
  'CCAG': позиции [7]
  'CTAG': позиции [0]
  'GATC': позиции [4]
  'GCAT': позиции [11]
  'GGAT': позиции [3]
  'GGCA': позиции [10]
  'TACG': позиции [14]
  'TAGG': позиции [1]
  'TCCA': позиции [6]

─── Найденные seeds (k=4) ───
  'GGAT':  query[0:4]  ==  ref[3:7]   стартовый score = 4
  'GATC':  query[1:5]  ==  ref[4:8]   стартовый score = 4
  'ATCC':  query[2:6]  ==  ref[5:9]   стартовый score = 4
  'TCCA':  query[3:7]  ==  ref[6:10]   стартовый score = 4


### Фаза 2: расширение (X-drop)

Расширяемся от границ seed в одну сторону, шаг за шагом:

- Стартовый `Scur = Smax = seed_score` 
- На каждом шаге добавляем `score_pair(query[q], ref[r])`
- Если новый `Scur > Smax` → обновляем `Smax` и запоминаем позицию
- Если `Smax − Scur ≥ X` → **стоп**, хвост отбрасываем

In [9]:
def extend(query, ref, q_start, r_start, direction, seed_score, X):
    """
    Расширяет seed в одну сторону (влево или вправо).

    Параметры
    q_start, r_start : стартовая позиция (левый или правый край seed)
    direction        : 'left' или 'right'
    seed_score       : начальный счёт (= k, т.к. seed совпал полностью)
    X                : порог X-drop

    Возвращает
    Smax   : лучший достигнутый счёт
    best_q : позиция в query, где был достигнут Smax
    best_r : позиция в ref,   где был достигнут Smax
    history: список шагов для вывода таблицы
    """
    Scur   = seed_score
    Smax   = seed_score
    best_q = q_start
    best_r = r_start
    history = []

    for step in range(1, max(len(query), len(ref)) + 1):
        # двигаемся на шаг в нужную сторону
        if direction == 'left':
            q = q_start - step
            r = r_start - step
        else:
            q = q_start + step
            r = r_start + step

        # вышли за границу
        if q < 0 or q >= len(query) or r < 0 or r >= len(ref):
            break

        # обновляем текущий счёт
        Scur += score_pair(query[q], ref[r])

        # если побили максимум — запоминаем позицию
        if Scur > Smax:
            Smax   = Scur
            best_q = q
            best_r = r

        history.append((step, q, r, query[q], ref[r], Scur, Smax))

        # X-drop
        if Smax - Scur >= X:
            break

    return Smax, best_q, best_r, history

In [10]:
best_result = None

for kmer, q_pos, r_pos in seeds:
    seed_score = k * MATCH          
    seed_end_q = q_pos + k - 1      
    seed_end_r = r_pos + k - 1     

    print(f"\n{'='*55}")
    print(f"  Seed '{kmer}':  query[{q_pos}:{q_pos+k}]  /  ref[{r_pos}:{r_pos+k}]")
    print(f"  Стартовый score = {seed_score}")
    print(f"{'='*55}")

    Smax_l, bq_l, br_l, hist_l = extend(
        query, reference, q_pos, r_pos, 'left', seed_score, X_DROP
    )
    print(f"\n  Расширение влево от query[{q_pos}] / ref[{r_pos}]:")
    print(f"  {'Шаг':>4}  {'q[i]':>5}  {'r[j]':>5}  {'q':>3}  {'r':>3}  {'Scur':>6}  {'Smax':>6}")
    print(f"  {'старт':>4}  {'—':>5}  {'—':>5}  {'—':>3}  {'—':>3}  {seed_score:>6}  {seed_score:>6}")
    for s, qi, ri, cq, cr, sc, sm in hist_l:
        print(f"  {s:>4}  {qi:>5}  {ri:>5}  {cq:>3}  {cr:>3}  {sc:>6}  {sm:>6}")
    print(f"  → Smax_left = {Smax_l}")

    Smax_r, bq_r, br_r, hist_r = extend(
        query, reference, seed_end_q, seed_end_r, 'right', seed_score, X_DROP
    )
    print(f"\n  Расширение вправо от query[{seed_end_q}] / ref[{seed_end_r}]:")
    print(f"  {'Шаг':>4}  {'q[i]':>5}  {'r[j]':>5}  {'q':>3}  {'r':>3}  {'Scur':>6}  {'Smax':>6}")
    print(f"  {'старт':>4}  {'—':>5}  {'—':>5}  {'—':>3}  {'—':>3}  {seed_score:>6}  {seed_score:>6}")
    for s, qi, ri, cq, cr, sc, sm in hist_r:
        print(f"  {s:>4}  {qi:>5}  {ri:>5}  {cq:>3}  {cr:>3}  {sc:>6}  {sm:>6}")
    print(f"  → Smax_right = {Smax_r}")

    total   = Smax_l + Smax_r - seed_score
    q_left  = min(bq_l, q_pos)
    q_right = max(bq_r, seed_end_q)
    r_left  = min(br_l, r_pos)
    r_right = max(br_r, seed_end_r)
    q_aln   = query[q_left     : q_right + 1]
    r_aln   = reference[r_left : r_right + 1]

    print(f"\n  Итог: Smax_left={Smax_l} + Smax_right={Smax_r} - seed={seed_score} = {total}")
    print(f"  query[{q_left}:{q_right+1}]:  {q_aln}")
    print(f"  ref  [{r_left}:{r_right+1}]:  {r_aln}")

    if best_result is None or total > best_result[0]:
        best_result = (total, kmer, q_pos, r_pos,
                       q_aln, r_aln, q_left, r_left, Smax_l, Smax_r)


  Seed 'GGAT':  query[0:4]  /  ref[3:7]
  Стартовый score = 4

  Расширение влево от query[0] / ref[3]:
   Шаг   q[i]   r[j]    q    r    Scur    Smax
  старт      —      —    —    —       4       4
  → Smax_left = 4

  Расширение вправо от query[3] / ref[6]:
   Шаг   q[i]   r[j]    q    r    Scur    Smax
  старт      —      —    —    —       4       4
     1      4      7    C    C       5       5
     2      5      8    C    C       6       6
     3      6      9    A    A       7       7
     4      7     10    T    G       6       7
     5      8     11    T    G       5       7
  → Smax_right = 7

  Итог: Smax_left=4 + Smax_right=7 - seed=4 = 7
  query[0:7]:  GGATCCA
  ref  [3:10]:  GGATCCA

  Seed 'GATC':  query[1:5]  /  ref[4:8]
  Стартовый score = 4

  Расширение влево от query[1] / ref[4]:
   Шаг   q[i]   r[j]    q    r    Scur    Smax
  старт      —      —    —    —       4       4
     1      0      3    G    G       5       5
  → Smax_left = 5

  Расширение вправо от query

In [11]:
total, best_km, bqp, brp, q_aln, r_aln, ql, rl, sl, sr = best_result

markup = ''.join('|' if a == b else '*' for a, b in zip(q_aln, r_aln))

print("  ЛУЧШЕЕ ВЫРАВНИВАНИЕ")
print(f"  Лучший k-мер : '{best_km}'")
print(f"  Smax_left    : {sl}")
print(f"  Smax_right   : {sr}")
print(f"  Итоговый Smax: {sl} + {sr} - {k} = {total}")
print(f"  query[{ql}:{ql+len(q_aln)}]:  {q_aln}")
print(f"  {'':>{9+len(str(ql))+len(str(ql+len(q_aln)))}}  {markup}")
print(f"  ref  [{rl}:{rl+len(r_aln)}]:  {r_aln}")
print(f"  Совпадений : {markup.count('|')}")
print(f"  Замен  : {markup.count('*')}")
print(f"  Референс:  {reference}")
print(f"  Позиция:   {'_'*rl}{'^'*len(r_aln)}")
print(f"             ref[{rl}:{rl+len(r_aln)}]")

  ЛУЧШЕЕ ВЫРАВНИВАНИЕ
  Лучший k-мер : 'GGAT'
  Smax_left    : 4
  Smax_right   : 7
  Итоговый Smax: 4 + 7 - 4 = 7
  query[0:7]:  GGATCCA
               |||||||
  ref  [3:10]:  GGATCCA
  Совпадений : 7
  Замен  : 0
  Референс:  CTAGGATCCAGGCATACGA
  Позиция:   ___^^^^^^^
             ref[3:10]


## Вывод

Все четыре seed (`GGAT`, `GATC`, `ATCC`, `TCCA`) перекрываются и дают одно и то же выравнивание с `Smax = 7`.

Лучший seed — **`GGAT`** (первый найденный, наибольшее расширение вправо).

Итоговое выравнивание:

```
query[0:7]:  GGATCCA
             |||||||
ref  [3:10]: GGATCCA
```

7 позиций, все совпадения — фрагмент `GGATCCA` полностью совпадает в обеих последовательностях.  
Расширение вправо остановилось по X-drop: после позиции `ref[9]` шли два мисматча подряд (`G≠T`, `G≠T`), и `Smax − Scur = 7 − 5 = 2 ≥ X`.